# Constraints en Pyomo

In [1]:
# Esta celda da el estilo al notebook
from IPython.core.display import HTML
css_folder = r'Styles_for_JN'
css_style = 'style_1.css'
css_file = css_folder + '\\' + css_style
HTML(open(css_file, "r").read())

## Importes

Se importa Pyomo: 

In [2]:
from pyomo.environ import *

## Declaraciones

El código siguiente proporciona la declaración más directa de una restricción:

In [3]:
model = ConcreteModel()
model.x = Var([1,2], initialize = 1)
model.diff = Constraint(expr = model.x[2] - model.x[1] <= 7.5)

In [4]:
model.diff.pprint()

diff : Size=1, Index=None, Active=True
    Key  : Lower : Body        : Upper : Active
    None :  -Inf : x[2] - x[1] :   7.5 :   True


Suele ser más cómodo utilizar una `rule` que una `expr`. Por ejemplo:

In [5]:
def diff_rule(model):
    return model.x[2] - model.x[1] <= 7.5
model.diff2 = Constraint(rule = diff_rule)
model.diff2.pprint()

diff2 : Size=1, Index=None, Active=True
    Key  : Lower : Body        : Upper : Active
    None :  -Inf : x[2] - x[1] :   7.5 :   True


Las restricciones pueden ser indexadas:

In [6]:
N = [1,2,3]
a = {1:1, 2:3.1, 3:4.5}
b = {1:1, 2:2.9, 3:3.1}

model.y = Var(N, within = NonNegativeReals, initialize = 0)

def CoverConstr_rule(model,i):
    return a[i] * model.y[i] >= b[i]
model.CoverConstr = Constraint(N, rule = CoverConstr_rule)
model.CoverConstr.pprint()

CoverConstr : Size=3, Index=CoverConstr_index, Active=True
    Key : Lower : Body       : Upper : Active
      1 :   1.0 :       y[1] :  +Inf :   True
      2 :   2.9 : 3.1 * y[2] :  +Inf :   True
      3 :   3.1 : 4.5 * y[3] :  +Inf :   True


Genera una constante para cada índice.

## Tipos de restricciones

Existen tres tipos de restricciones en Pyomo:

#### 1) Restricciones de desigualdad

Tienen la forma:
$$expr_{1} \leq expr_{2} \qquad or\qquad expr_{1} \geq expr_{2}$$

No se soporta > o <.

#### 2) Restricciones de igualdad

Tienen la forma:
$$ expr_{1} = expr_{2}$$

#### 3) Restricciones de rango

Tienen la forma:
$$ lb \leq expr_{1} \leq ub$$

En algunos modelos de optimización, una restriccion puede no ser definida para todoslos índices. En estos casos se usa `Constraint.Skip`.

In [7]:
model.TimePeriods = Set(initialize = [1,2,3,4,5], ordered = True)

In [8]:
LastTimePeriod = model.TimePeriods[-1]
model.StartTime = Var(TimePeriods, initialize = 1)
def Pred_rule(model,t):
    if t == LastTimePeriod:
        return Constraint.Skip
    else:
        return model.StartTime[t] <= model.StartTime[t+1]
model.Pred = Constraint(model.TimePeriods, rule = Pred_rule)
model.Pred.pprint()

NameError: name 'TimePeriods' is not defined

## Extracción de información de restricciones

A continuación, se dan algunos comandos para conocer información de una restriccion:

In [ ]:
m = ConcreteModel()
m.x = Var(initialize = 1)
m.y = Var(initialize = 1)
m.c1 = Constraint(expr = m.y - m.x <= 7.5)
m.c2 = Constraint(expr = -2.5 <= m.y - m.x)
m.c3 = Constraint(expr = -3 <= m.y - m.x <= 7)

In [ ]:
print(value(m.c1.body))

In [ ]:
print(m.c1.body)

In [ ]:
print(m.c1.lslack())
print(m.c1.uslack())